<a href="https://colab.research.google.com/github/OneFineStarstuff/OneFineStarstuff/blob/main/Adversarial_Training_Example_with_FGSM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset

# Example data (replace with your actual data)
X_train = torch.randn(100, 3, 32, 32)  # 100 samples, 3 channels, 32x32 image
y_train = torch.randint(0, 10, (100,))  # 100 labels, for 10 classes

# Create dataset and dataloader
dataset = TensorDataset(X_train, y_train)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

# Define your model (example model)
class MyModel(torch.nn.Module):
    def __init__(self):
        super(MyModel, self).__init__()
        self.conv1 = torch.nn.Conv2d(3, 10, kernel_size=5)
        self.fc1 = torch.nn.Linear(10*28*28, 10)  # Assuming input size 32x32

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = x.view(-1, 10*28*28)
        x = self.fc1(x)
        return F.log_softmax(x, dim=1)

model = MyModel()
optimizer = torch.optim.Adam(model.parameters())

def fgsm_attack(model, inputs, labels, epsilon=0.1):
    inputs.requires_grad = True
    outputs = model(inputs)
    loss = F.nll_loss(outputs, labels)
    model.zero_grad()
    loss.backward()
    perturbed_inputs = inputs + epsilon * inputs.grad.sign()
    return perturbed_inputs

# Train on adversarial examples
for data, labels in dataloader:
    optimizer.zero_grad()  # Clear gradients
    adv_data = fgsm_attack(model, data, labels)
    output = model(adv_data)
    loss = F.cross_entropy(output, labels)
    loss.backward()
    optimizer.step()